### Imports

In [1]:
from __future__ import annotations

from pathlib import Path

from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.simlib.simulation import FMUSimulation

# Find repository root: check if database/ exists at current location
repo_root = (
    Path.cwd() if (Path.cwd() / "database").exists() else Path.cwd().parent.parent
)
databasePath = repo_root / "database" / "day1" / "FMUs"

In [2]:
print(repo_root)
print(databasePath)

/home/bjanssens/src/courses/building_energy/2025-course-kub-barche
/home/bjanssens/src/courses/building_energy/2025-course-kub-barche/database/day1/FMUs


### **Conduction: Exercise 1**

<img src="../../database/images/Conduction1.png"
     align="right"
     width="550"
     style="margin-right: 20px; margin-bottom: 10px;" />

<ins>**Scenario 1: Wall_Resistor (Determine Resistance R)**<ins>

The component Wall_Resistor is subjected to the following conditions:
- Interior Temperature &emsp; $(T_{int,R}​)$ &emsp; : &emsp; $20$ °C
- Exterior Temperature &emsp; $(T_{ext})$ &emsp; : &emsp; $5$ °C
- Heat Flow &emsp; &emsp; &emsp; &emsp; &emsp; $(Φ_R​)$ &emsp; : &emsp; $10$ W

**Goal:** Calculate the Thermal Resistance $(R)$ of the Wall_resistor.

<ins>**Scenario 2: Wall_Conductor (Determine Conductance U​)**<ins>

The component Wall_Conductor is subjected to the following conditions:
- Interior Temperature &emsp; $(T_{int,U})$ &emsp; : &emsp; $25$ °C
- Exterior Temperature &emsp; $(T_{ext}​)$ &emsp; : &emsp; $5$ °C
- Heat Flow &emsp; &emsp; &emsp; &emsp; &emsp; $(Φ_U)$ &emsp; : &emsp; $15$ W

**Goal:** Calculate the Thermal Conductance $(U)$ of the Wall_resistor.

In [3]:
sim1 = FMUSimulation(databasePath / "Exercises_Conduction_SteStaWalResCon_01.fmu")

sim1.initialize()
sim1.initParameters({"Wall_Resistor.R": 1.5, "Wall_Conductor.G": 0.75})
sim1.exitInitialization()

data1 = sim1.run(["Wall_Resistor.port_b.T", "Wall_Conductor.port_b.Q_flow"])
sim1.terminate()

sim1Factory = SimulationPlotFactory()

sim1Factory.plot_subplots(
    time=data1["time"],
    data_type="raw",
    data_dict={
        "Wall Resistor Temperature (°C)": data1["Wall_Resistor.port_b.T"] - 273.15,
        "Wall Conductor Heat Flow (W)": data1["Wall_Conductor.port_b.Q_flow"],
    },
    title="Exercise 1 — Output variables",
)

### **Conduction: Exercise 2**

<img src="../../database/images/Conduction2.png"
     align="right"
     width="550"
     style="margin-right: 20px; margin-bottom: 10px;" />

<ins>Boundary conditions:<ins>

- Exterior temperature of : 5°C (278.15 K)
- wall_resistor R value : 3 K/W
- wall_conductor U value : 0.2 W/K

<ins>Unknowns :<ins>
- Interior Heat flow : Q_flow

<ins>Expected result :<ins>
- Room air : temperature of 20°C


In [8]:
q_ex_2 = 1/3*15+0.2*15

In [9]:
sim2 = FMUSimulation(databasePath / "Exercises_Conduction_SteStaWalResCon_02.fmu")

sim2.initialize()
sim2.initParameters({"InteriorHeatFlow.Q_flow": q_ex_2})
sim2.exitInitialization()

data2 = sim2.run(["RoomAir.port.T"])
sim2.terminate()

sim2Factory = SimulationPlotFactory()

sim2Factory.plot_subplots(
    time=data2["time"],
    data_type="temperature",
    data_dict={"Room Air Temperature (°C)": data2["RoomAir.port.T"]},
    title="Exercise 2 — Room Air Temperature",
)

### **Conduction: Exercise 3**

You are tasked with determining the necessary heating power $(P)$ required to maintain a specific interior temperature for a composite wall.

<img src="../../database/images/Conduction3.png"
     align="right"
     width="400"
     style="margin-right: 20px; margin-bottom: 10px;" />

**1. Wall Specifications**

The wall has a total surface area $(A)$ of $1$ square meter $(m²)$ and is composed of three distinct material sections: **Wood**, **Earth**, and **Straw**. \
The wall has a non-uniform structure, suggesting a combination of series and parallel heat transfer paths.

**2. Thermal Boundary Conditions**

- Desired Interior Temperature &nbsp; $(T_{in}​$) &nbsp; : &nbsp; $20$°C
- Exterior Temperature &nbsp; $(T_{out}​$) &nbsp; : &nbsp; $5$°C

**3. Objective**

Calculate the heating power $(P)$ in Watts $(W)$ required to compensate for the heat loss through the wall, thus maintaining the interior temperature at $T_{in}​=20$°C.

$Q = R_{total​} *(T_{in}​−T_{out}​)$ &emsp; &emsp; Where $R_{total}$​ is the total thermal resistance of the composite wall, that you will need to calculate.

Material | Thermal Conductivity (𝛌)
---------|-------------------------
Wood     | 0.13 W/m/K
Earth coating | 0.75 W/m/K
Straw | 0.07 W/m/K

In [ ]:
sim3 = FMUSimulation(databasePath / "Exercises_Conduction_ComWalSys_03.fmu")

sim3.initialize()

wallParameters = {
    "wall_1.lambda": 0.75,
    "wall_1.e": 0.05,
    "wall_1.A": 0.7,
    "wall_2.lambda": 0.07,
    "wall_2.e": 0.15,
    "wall_2.A": 0.7,
    "wall_3.lambda": 0.13,
    "wall_3.e": 0.20,
    "wall_3.A": 0.3,
    "wall_4.lambda": 0.75,
    "wall_4.e": 0.05,
    "wall_4.A": 0.7,
}
sim3.initParameters(wallParameters)
sim3.initParameters({"InteriorHeatFlow.Q_flow": 7.35})

sim3.exitInitialization()

data3 = sim3.run(["wall_4.port_b.T", "InteriorHeatFlow.port.T"])
sim3.terminate()

sim3Factory = SimulationPlotFactory()

sim3Factory.plot_subplots(
    time=data3["time"],
    data_type="temperature",
    data_dict={"Room Air Temperature (°C)": data3["wall_4.port_b.T"]},
    title="Exercise 3 — Room Air Temperature",
)

### **Conduction: Exercise 4**

<img src="../../database/images/Conduction4.png"
     align="right"
     width="600"
     style="margin-right: 20px; margin-bottom: 10px;" />

This problem requires an analysis and comparison of the dynamic thermal behavior of two $1$ $m^2$ composite wall, assessing the impact of insulation placement (internal vs. external) under transient temperature loads.

The walls in question are composed of two layers, each $5$ $cm$ thick, with the following material properties:

Component | Thermal Conductivity $(λ)$ | Thickness $(e)$ | Volumetric Heat Capacity $(ρ*cp​)$ | Total Heat Capacity $(C)$ for $1$ $m^2$
-|-|-|-|-|
Concrete | $0.13$ $W/m/K$ | $0.05$ $m$ | $10 112$ $kJ/(m^3K)$ | $505 600$ $J/K$
Glass Wool | $0.034$ $W/m/K$ | $0.05$ $m$ | $25,75$ $kJ/(m^3K)$ | $1 287,5$ $J/K$

**Simulation Conditions:**

- **Initial Condition:** The entire wall is at a uniform temperature of $0$°C at the start of the simulation $(t=0)$.
- **External Boundary Condition:** The exterior temperature $(T_{ext}​)$ start at $10$°C at $t=0$, oscillating between $0$°C and $20$°C (e.g., following a daily sinusoidal cycle).
- **Internal Boundary Condition:** The internal environment is allowed to vary freely.

What **differences** in thermal behavior : **temperature profiles**, **time lag**, and **dampening of thermal waves** can be observed between the two models under these dynamic conditions ?


In [ ]:
sim4 = FMUSimulation(databasePath / "Exercises_Conduction_ComWalSys_04.fmu")

sim4.initialize(startTime=0.0, stopTime=86400 * 30, timeStep=3600.0)
sim4.exitInitialization()

data4 = sim4.run(
    [
        "ExteriorInsulationTemperatureSensor.T",
        "InteriorInsulationTemperatureSensor.T",
        "ExteriorTemperature.T",
    ]
)
sim4.terminate()

sim4Factory = SimulationPlotFactory()

sim4Factory.plot_multi_curves(
    time=data4["time"],
    data_type="temperature",
    data_dict={
        "Interior Temperature with Exterior Insulation (°C)": data4[
            "ExteriorInsulationTemperatureSensor.T"
        ],
        "Interior Temperature with Interior Insulation (°C)": data4[
            "InteriorInsulationTemperatureSensor.T"
        ],
        "Exterior Temperature (°C)": data4["ExteriorTemperature.T"],
    },
    title="Exercise 4 — Type of Insulation (Exterior vs Interior)",
)